In [39]:
from pymilvus import model
from pymilvus import MilvusClient, DataType

### Create Milvus connection to a file

In [27]:
client = MilvusClient("/tmp/milvus_demo.db")

client.drop_collection(collection_name="my_sparse_collection")

schema = client.create_schema(
    auto_id=True,
    enable_dynamic_fields=True,
)

schema.add_field(field_name="pk", datatype=DataType.VARCHAR, is_primary=True, max_length=100)
schema.add_field(field_name="id", datatype=DataType.VARCHAR, is_primary=False, max_length=100)
schema.add_field(field_name="embeddings", datatype=DataType.SPARSE_FLOAT_VECTOR)

index_params = client.prepare_index_params()

index_params.add_index(field_name="embeddings",
                               index_name="sparse_inverted_index",
                               index_type="SPARSE_INVERTED_INDEX",
                               metric_type="IP",
                               params={"drop_ratio_build": 0.2})
client.create_collection(
    collection_name="my_sparse_collection",
    schema=schema,
    index_params=index_params
)


In [28]:
embeddings_model = model.sparse.SpladeEmbeddingFunction(
    model_name="ibm-granite/granite-embedding-30m-sparse",
    device="cpu",
    batch_size=2,
    k_tokens_query=50,
    k_tokens_document=192
)

In [29]:
import os
os.getenv('CUDA_VISIBLE_DEVICES')

'1'

### Prepare documents to be ingested

In [30]:
docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]
vecs = embeddings_model.encode_documents(docs)

In [34]:
vecs.shape[1]

50265

In [36]:
dim = vecs.shape[1]
doc_vector = [{"embeddings": doc_emb.reshape(1,dim), "id": f"item_{i}"} for i, doc_emb in enumerate(vecs)]

client.insert(
    collection_name="my_sparse_collection",
    data=doc_vector
)


{'insert_count': 3, 'ids': ['456313206239133708', '456313206239133709', '456313206239133710'], 'cost': 0}

### Prepare search parameters

In [37]:
search_params = {
    "params": {"drop_ratio_search": 0.2},  # Additional optional search parameters
}


### Prepare the query vector

In [38]:

queries = [
      "When was artificial intelligence founded",
      "Where was Turing born?"
]
query_vector = embeddings_model.encode_documents(queries)

res = client.search(
    collection_name="my_sparse_collection",
    data=query_vector,
    limit=1, #top k documents to return
    output_fields=["id"],
    search_params=search_params,
)

for r in res:
    print(r)

[{'id': '456313203549011974', 'distance': 12.3641357421875, 'entity': {'id': 'item_0'}}]
[{'id': '456313203549011976', 'distance': 17.1358642578125, 'entity': {'id': 'item_2'}}]
